In [1]:
%%python
from importlib.util import find_spec
import shutil
import subprocess

fix = """
-------------------------------------------------------------------------
fix following the steps here:
    https://github.com/modularml/mojo/issues/1085#issuecomment-1771403719
-------------------------------------------------------------------------
"""

def install_if_missing(name: str):
    if find_spec(name):
        return

    print(f"{name} not found, installing...")
    try:
        if shutil.which('python3'): python = "python3"
        elif shutil.which('python'): python = "python"
        else: raise ("python not on path" + fix)
        subprocess.check_call([python, "-m", "pip", "install", name])
    except:
        raise ImportError(f"{name} not found" + fix)

install_if_missing("spacy")

In [2]:
from python import Python
let spacy = Python.import_module("spacy")

In [41]:
from math import sqrt

In [50]:
let nlp = spacy.load('en_core_web_lg')
let doc = nlp("dog flower tree.")

In [51]:
print(doc[0].vector.shape)

(300,)


In [58]:
fn numpy_to_simd[np_size: Int, size: Int](np_array: PythonObject) raises -> SIMD[DType.float64, size]:
    var result = SIMD[DType.float64, size]()
    for i in range(np_size):
        result[i] = np_array[i].to_float64()
    return result

In [44]:
fn cosine_similarity[size: Int](a: SIMD[DType.float64, size], b: SIMD[DType.float64, size]) -> Float64:
    return (a*b).reduce_add() / sqrt((a**2).reduce_add() * (b**2).reduce_add() )

In [64]:
%%python

import spacy
import numpy as np

nlp = spacy.load('en_core_web_lg')
doc = nlp("dog flower tree.")

a = doc[0].vector
b = doc[1].vector

dot_product = np.dot(a, b)
norm_a = np.linalg.norm(a)
norm_b = np.linalg.norm(b)

cosine_similarity = dot_product / (norm_a * norm_b)

print("Cosine Similarity:", cosine_similarity)

Cosine Similarity: 0.13728142


In [65]:


let v1 = numpy_to_simd[300, 512](doc[0].vector)
let v2 = numpy_to_simd[300, 512](doc[1].vector)
let v3 = numpy_to_simd[300, 512](doc[2].vector)

print(cosine_similarity(v1, v2))
print(cosine_similarity(v2, v1))
print(cosine_similarity(v1, v))

error: Expression [65]:7:24: invalid call to '__call__': argument #1 cannot be converted from 'SIMD[f64, 300]' to 'PythonObject'
print(cosine_similarity(v1, v2))
      ~~~~~~~~~~~~~~~~~^~~~~~~~

Expression [0]:1:1: function declared here
var __lldb_repl_python__ = __mojo_repl_Python()
^

error: Expression [65]:8:24: invalid call to '__call__': argument #1 cannot be converted from 'SIMD[f64, 300]' to 'PythonObject'
print(cosine_similarity(v2, v1))
      ~~~~~~~~~~~~~~~~~^~~~~~~~

Expression [0]:1:1: function declared here
var __lldb_repl_python__ = __mojo_repl_Python()
^

error: Expression [65]:9:24: invalid call to '__call__': argument #1 cannot be converted from 'SIMD[f64, 300]' to 'PythonObject'
print(cosine_similarity(v1, v1))
      ~~~~~~~~~~~~~~~~~^~~~~~~~

Expression [0]:1:1: function declared here
var __lldb_repl_python__ = __mojo_repl_Python()
^

expression failed to parse (no further compiler diagnostics)